<a href="https://colab.research.google.com/github/leinaxd/Tps/blob/master/a2/a2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS224N Assignment 2: word2vec (44 Points)

## PARTE 1 Escrito (26 points)
Refresquemos rápidamente el algoritmo word2vec. La intuición clave detrás de word2vec es que ‘Se conoce a una palabra por la compañia que tiene’.

Nos referiremos a la palabra central como $c$ y a las palabras vecinas a ella en la ventana de contexto como $o$. Por ejemplo, en la figura se observa que la palabra central $c$ es 'banking’. ya que el tamaño de la ventana de contexto es 2, las palabras de afuera son ‘turning’, ‘into’, ‘crises’, and ‘as’.

El objetivo del algoritmo word2vec skip-gram es el de aprender la distribución de probabilidad $P(O=o|C=c)$.  $P(O|C)$ es la probabilidad de que una palabra sea del contexto $o$ para $c$.

![word2vec](https://github.com/leinaxd/Tps/blob/master/a2/imgs/word2vec_prob.png?raw=1)

En word2vec, la distribución de probabilidad condicional esta definida con el producto intero y la función softmax.

$$ P(O = o | C = c) = \frac{exp(u_o^T v_c)}{\sum_{w\in vocab}exp(u_w^T v_c)} $$ (1)

Aquí, $u_o\in \rm I\!R^{K}$ es el vector de 'la ventana de contexto' representando la palabra 'o' de la ventana, y $v_c\in \rm I\!R^{K}$ es el vector del 'centro' representando la palabra del centro $c$.
Para almacenar estos parámetros, definimos dos matrices, $U\in \rm I\!R^{K\times |V|}$ y $V\in \rm I\!R^{K\times |V|}$.
Las columnas de $U$ son todos los vectores $u_w$ del 'contexto'.
Las columnas de $V$ son todos los vectores $v_w$ del 'centro'.
Ambos $U$ y $V$ contienen un vector para cada $w\in Vocabulary$

Recordar de las clases que, para un par de palabras $c$ y $o$ la función de costo está dada por:

$$J_{naive-softmax}(v_c, o, U) = -log\ P(O=o|C=c) $$ (2)

Además, la entropía cruzada entre una distribución de probabilidad verdadera $p$ y otra distribución $q$ para una variable discreta $x$ era:

$$ H(p,q) = -\sum_{x\in X}p(x)\ log(q(x)) $$ <br>

Se puede ver este coste como la entropía cruzada entre la verdadera distribución de $y$ y la distribución estimada $\hat y$.
Aquí, tanto $y$ como $\hat y$ son vectores de igual longitud al número de palabras del vocabulario. Además, la componente $k^{esima}$ de estos vectores indican la probabilidad condicional de que la $k^{esima}$ palabra sea una palabra de 'contexto' dada la palabra 'c'.
La verdadera distribución empírica de $y$ es un *one-hot vector* con un 1 para la verdadera palabra 'de contexto' y 0 en otro caso. La distribución estimada $\hat y$ es la distribución de probabilidad $P(O|C = c)$ Dado nuestro modelo de la ecuación (1).

## PARTE 1a (3 points)
Mostrar que la función de costo Naive-Softmax coincide con la función de coste de Cross-Entropy entre $y$ y $\hat{y}$

$$J=-\sum_{w\in Vocab} y_w\ log\ (\hat y_w) = -log(\hat y_o)$$

La respuesta debe ser una línea

<font color="green">Respuesta:<br>
$\begin{align}
H(P,\hat P) &= -\sum_{w\in vocab} P(O|C)\ log(\hat P(O|C))  &&
\small\text{La prob. que una palabra sea del contexto de c}\\
H(y,\hat y)&= -\sum_{i=1\atop \text{Coord. vector}}^K y_i\ log(\hat y_i)&&
\small\text{Representar en One-Hot encoding no cambia la prob.}\\
&=-1_{i = i_o}\ log(\hat y_o) - 0_{i\neq i_o}\ log(\hat y_i)&&
\small\text{One-Hot = 1 para la componente del vector correcta}\\
&=-log(\hat y_o)
\end{align}$
</font>

## PARTE 1b (5 points)
Calcular la derivada parcial de la función $J_{naive-softmax}(v_c, o, U)$ respecto a $v_c$.
Escribir la respuesta en términos de $y$, $\hat y$ y $U$.
<br><br><br>

·Se espera que las respuestas finales cumplan la **convención de dimensión**. Esto significa que la derivada parcial de cualquier función $f(x)$ respecto a $x$ debe tener la misma dimensión que $x$.

·Presentar la respuestas en su **forma vectorizada**. (No deberías referirte a elementos específicos de $y$, $\hat y$ o $U$ como $y_1$, $y_2$, ...)

Esto nos permite minimizar eficientemente una función sin preocuparnos por redimensionar las matrices al utilizar el gradiente descendiente.
Utilizando esta convención, nos garantizamos que la regla de actualización
$θ := θ − α \frac{\partial J(θ)}{\partial \theta}$ esté bien definida.

<font color="green">Respuesta: <br>
$\begin{align}
J_{naive-softmax}(v_c, o, U) &= -log\ \hat P(O=o|C=c)\\
&=-log\ \frac{exp(u_o^T v_c)}{\sum_{w\in vocab}exp(u_w^T v_c)}\\
&= -u_o^T v_c\ +\ log\ \sum_{w\in vocab}exp(u_w^T v_c)
\end{align}$
<br>
$\begin{align}
\frac{\partial}{\partial v_c} J_{naive-softmax}(v_c, o, U)
&= \frac{\partial}{\partial v_c} -u_o^T v_c\ + \ \frac{\partial}{\partial v_c}\ log \sum_{w\in vocab}exp(u_w^T v_c)\\
&=-u_o\ +\ \frac{1}{\sum_{w\in vocab}exp(u_w^T v_c)}\sum_{w\in vocab}
\begin{bmatrix}
    \frac{\partial}{\partial v_c^1}exp(u_w^T v_c)\\
    \vdots\\
    \frac{\partial}{\partial v_c^K}exp(u_w^T v_c)\\
\end{bmatrix} \\\
&=-u_o\ +\ \frac{1}{\sum_{w\in vocab}exp(u_w^T v_c)}\sum_{w\in vocab}
\begin{bmatrix}
    exp(u_w^T v_c)\ ·\ u_w^1\\
    \vdots\\
    exp(u_w^T v_c)\ ·\ u_w^K\\
\end{bmatrix} \\\
\end{align}
$

$\begin{align}
&=-u_o\ +\ \frac{
\sum_{w\in vocab}
exp(u_w^T v_c)\ ·\ u_w
}{\sum_{w\in vocab}exp(u_w^T v_c)}\\\
&=-u_o\ +\ \sum_{w\in vocab} u_w \frac{
exp(u_w^T v_c)
}{\sum_{w'\in vocab}exp(u_{w'}^T v_c)}\\\
&=-u_o\ +\
\underbrace{U}_{\in \mathbb{R}^{K\times |V|}} ·\
\underbrace{softmax(U^T v_c)}_{\in \mathbb{R}^{|V|\times 1}}\\
&=-U·y + U · \hat y\\
&=U · (\hat y - y)
\end{align}
$

Donde La función Softmax es:

$\begin{align}
\sigma (
    \begin{bmatrix}
        z_1 \\
        \vdots \\
        z_K
    \end{bmatrix} )=
    \begin{bmatrix}
        \frac {z_1}{\sum_{j=1}^K exp(z_j)} \\
        \vdots \\
        \frac {z_K}{\sum_{j=1}^K exp(z_j)}
    \end{bmatrix}
\end{align}$

Donde:

$\hat y = \begin{bmatrix}
            \hat y_1 \\
            \vdots \\
            \hat y_K
         \end{bmatrix} = softmax(U^T v_c) = softmax(U^T V x_{one-hot})$

$y = Enc_{one-hot} (w_i)$ (Muestras supervisadas)

</font>

## PARTE 1c (5 points)
Calcular la derivada parcial de $J_{naive-softmax}(v_c, o, U)$ respecto a cada word vector de salida $u_w$. <br>
Habrá dos casos: cuando $w=o$ sea el verdadero vector de salida
y cuando $w\neq o$ para el resto de palabras

·Escriba su respuesta en términos de $y$, $\hat y$ y $v_c$. En esta parte podrás utilizar elementos específicos como $y_1$, $y_2$, etc.

<font color="green">Respuesta:
<br>
$J_{naive-softmax}(v_c, o, U) = -u_o^T v_c\ +\ log\ \sum_{w\in vocab}exp(u_w^T v_c)$
<br>
Para $w = o$:
<br>
$\begin{align}
\frac{\partial}{\partial u_o} J_{naive-softmax}(v_c, o, U)
&= \frac{\partial}{\partial u_o} -u_o^T v_c\ + \ \frac{\partial}{\partial u_o}\ log \sum_{w\in vocab}exp(u_w^T v_c)\\
&= -v_c\ + \frac{1}{\sum_{w\in vocab}exp(u_w^T v_c)}
\ \frac{\partial}{\partial u_o}\ \sum_{w\in vocab} exp(u_w^T v_c)\\
&= -v_c\ + \frac{1}{\sum_{w\in vocab}exp(u_w^T v_c)}
\ \frac{\partial}{\partial u_o}\ exp(u_o^T v_c)\\
&= -v_c\ + \frac{1}{\sum_{w\in vocab}exp(u_w^T v_c)}
\begin{bmatrix}
    \frac{\partial}{\partial u_o^1}\ exp(u_o^T v_c)\\
    \vdots\\
    \frac{\partial}{\partial u_o^K}\ exp(u_o^T v_c)\\
\end{bmatrix}\\\
&= -v_c\ + \frac{1}{\sum_{w\in vocab}exp(u_w^T v_c)}
\begin{bmatrix}
    exp(u_o^T v_c)\ v_c^1\\
    \vdots\\
    exp(u_o^T v_c)\ v_c^K\\
\end{bmatrix}\\\
&= -v_c\ + \frac{ exp(u_o^T v_c)\ v_c }{\sum_{w\in vocab}exp(u_w^T v_c)}\\\
&= (softmax^T(U^T v_c)·y_o-1)v_c\\\
&= (\hat y^T·y-1)v_c\\\
\end{align}$
<br>
Para $w \neq o$:
<br>
$\begin{align}
\frac{\partial}{\partial u_w} J_{naive-softmax}(v_c, o, U)
&= \frac{\partial}{\partial u_w} -u_o^T v_c\ + \ \frac{\partial}{\partial u_w}\ log \sum_{w'\in vocab}exp(u_{w'}^T v_c)\\
&= \frac{1}{\sum_{w'\in vocab}exp(u_{w'}^T v_c)} \frac{\partial}{\partial u_w}\ \sum_{w'\in vocab}exp(u_{w'}^T v_c)\\
&= \frac{1}{\sum_{w'\in vocab}exp(u_{w'}^T v_c)} \frac{\partial}{\partial u_w}\ exp(u_w^T v_c)\\
&= \frac{exp(u_w^T v_c)\ v_c}{\sum_{w'\in vocab}exp(u_{w'}^T v_c)}\\
&= softmax^T(U^T\ v_c)·y_w\ v_c\\
&= \hat y^T·y_w\ v_c\\
\end{align}$

</font>

## PARTE 1d (1 point)
Calcular la derivadad parcial de $J_{naive-softmax}(v_c,o,U)$ respecto a $U$

·Escribir su respuesta en términos de $\frac{\partial J(v_c, o, U)}{\partial u_1}$, $\frac{\partial J(v_c, o, U)}{\partial u_2}$, ..., $\frac{\partial J(v_c, o, U)}{\partial u_{|Vocab|}}$

·La solución debe tener una o dos líneas

<font color="green">Respuesta:<br>
$\begin{align}
\frac{\partial}{\partial U} J_{naive-softmax}
&=\begin{bmatrix}
    \frac{\partial}{\partial u_1} J &
    \frac{\partial}{\partial u_2} J &
    \cdots &
    \frac{\partial}{\partial u_o} J &
    \cdots &
    \frac{\partial}{\partial u_{|V|}}
  \end{bmatrix}\\\
&=\begin{bmatrix}
     \frac{exp(u_1^T v_c)\ v_c}{\sum_{w'\in vocab}exp(u_{w'}^T v_c)} &
     \cdots
     (-v_c\ + \frac{ exp(u_o^T v_c)\ v_c }{\sum_{w\in vocab}exp(u_w^T v_c)})&
     \cdots &
     \frac{exp(u_{|V|}^T v_c)\ v_c}{\sum_{w'\in vocab}exp(u_{w'}^T v_c)}
  \end{bmatrix}\\\
&=
\begin{bmatrix}
     0&
     \cdots
     -v_c&
     \cdots &
     0
\end{bmatrix}+v_c
\odot
\ \underbrace{softmax^T(U^T ·v_c)}_{\mathbb{R}^{1\times |V|}}
\end{align}$
</font>

## PARTE 1e (3 Points)
La función sigmoidea es<br>
$$σ(x) = \frac{1}{1 + e^{−x}} = \frac{e^x}{e^x + 1} $$<br>
Obtener la derivada de $σ(x)$ respecto a $x$, donde $x$ es un escalar.<br>
Pista: Querrás reescribir tu respuesta en términos de $σ(x)$.

<font color="green">Respuesta:<br>
$\begin{align}
\frac{\partial}{\partial x} \sigma (x)
&=\frac{e^x}{e^x+1}-\frac{e^x\ e^x}{(e^x+1)^2}\\
&=\frac{e^{x}}{e^x+1}\ ·\ (1-\frac{e^x}{e^x+1})\\
&=\sigma(x)\ ·\ (1-\sigma(x))
\end{align}$
</font>

## PARTE 1f (4 points)
Ahora consideraremos la función de coste de **Negative Sampling** que es una alternativa a la función de coste Naive softmax.<br>
Suponer que se toman del vocabulario $\tilde K$ muestras negativas (palabras).
Por sencillez de notación, nos referiremos a ellas por $\tilde w_1, \tilde w_2, ..., \tilde w_K$
Y a sus vectores del decodificador (contexto) como $\tilde u_1, ..., \tilde u_K$.

Para esta pregunta, suponer que las $\tilde K$ muestras son diferentes, en otras palabras, $\tilde w_i\neq \tilde w_j$ para $i\neq j \wedge i,j\in\{1,...,K\}$. Observar que $o\neq\{\tilde w_1, ..., \tilde w_K\}$.

Para una palabra central $c$ y una palabra externa $o$, la función de costo de Negative-Sampling es

$$ J_{neg-sample}(v_c, o, U) = -log(σ(u_o^T v_c))−\sum_{k=1}^{\tilde K} log(σ(−\tilde u_k^T v_c)) $$

* Repetir los ejercicios 1b y 1c, calculando las derivadas parciales de $J_{neg-sample}$ respecto a $v_c$, respecto a $u_o$ y respecto la muestra negativa $\tilde u_k$.<br>
* Escribir las respuestas en términos de vectores de $u_o$, $v_c$ y $\tilde u_k$ donde $k\in [1,\tilde K]$

* Al finalizar, escribir en una oración, por qué esta función de coste es mucho más eficiente de calcular que la función Naive-Softmax.

·Obs. Debes ser capaz de utilizar la solución del 1e para calcular los gradientes requeridos.

<font color="green">Respuesta:<br>
$\begin{align}
\frac{\partial}{\partial v_c} J_{neg.\ sample}(v_c,o,U)
&=- \frac{\partial}{\partial v_c}
 log(\sigma(u_o^T v_c))
 -\frac{\partial}{\partial v_c}
 \sum_{k=1}^{\tilde K} log(\sigma(-\tilde u_k^T v_c))\\
&=- \frac{1}{\sigma(u_o^T v_c)}
 \frac{\partial}{\partial v_c}
 \sigma(u_o^T v_c)
 -\sum_{k=1}^{\tilde K}
 \frac{\partial}{\partial v_c}
 log(\sigma(-\tilde u_k^T v_c))\\
&=- \frac{\sigma(u_o^T v_c) · (1 - \sigma(u_o^T v_c))}
 {\sigma(u_o^T v_c)}
 \frac{\partial}{\partial v_c}
 u_o^T v_c
 -\sum_{k=1}^{\tilde K}
 \frac{1}{\sigma(-\tilde u_k^T v_c)}
 \frac{\partial}{\partial v_c}
 \sigma(-\tilde u_k^T v_c)\\
&=- (1 - \sigma(u_o^T v_c))\ u_o
 -\sum_{k=1}^{\tilde K}
 \frac{ \sigma(-\tilde u_k^T v_c) (1-\sigma(-\tilde u_k^T v_c))}{\sigma(-\tilde u_k^T v_c)}
 \frac{\partial}{\partial v_c}
 -\tilde u_k^T v_c\\
&=- (1 - \sigma(u_o^T v_c))\ u_o
 +\sum_{k=1}^{\tilde K}
 (1-\sigma(-\tilde u_k^T v_c)) \tilde u_k\\
\end{align}$

$\begin{align}
\frac{\partial}{\partial u_o} J_{neg.\ sample}(v_c,o,U)
&=- \frac{\partial}{\partial u_o}
 log(\sigma(u_o^T v_c))
 -\frac{\partial}{\partial u_o}
 \sum_{k=1}^{\tilde K} log(\sigma(-\tilde u_k^T v_c))\\
&=-\frac{1}{\sigma(u_o^T v_c)}
 \frac{\partial}{\partial u_o} \sigma(u_o^T v_c) -0\\
&=-\frac{\sigma(u_o^T v_c)(1-\sigma(u_o^T v_c))}{\sigma(u_o^T v_c)}
 \frac{\partial}{\partial u_o} (u_o^T v_c)\\
&=-(1-\sigma(u_o^T v_c))\ v_c\\
\end{align}$

$\begin{align}
\frac{\partial}{\partial \tilde u_k} J_{neg.\ sample}(v_c,o,U)
&=- \frac{\partial}{\partial \tilde u_k}
 log(\sigma(u_o^T v_c))
 -\frac{\partial}{\partial \tilde u_k}
 \sum_{k'=1}^{\tilde K} log(\sigma(-\tilde u_{k'}^T v_c))\\
&=- 0 - \frac{\partial}{\partial \tilde u_k}
 log(\sigma(-\tilde u_{k}^T v_c))\\
&= - \frac{1}{\sigma(-\tilde u_k^T v_c)}
 \frac{\partial}{\partial \tilde u_k} \sigma(-\tilde u_k^T v_c)\\
&= -\frac{\sigma(-\tilde u_k^T v_c)(1-\sigma(-\tilde u_k^T v_c))}{\sigma(-\tilde u_k^T v_c)}
 \frac{\partial}{\partial \tilde u_k} (-\tilde u_k^T v_c)\\
&= (1-\sigma(-\tilde u_k^T v_c))\ v_c\\
\end{align}$
</font>

<font color="00B000">Es más eficiente que Naive Softmax en la evaluación de los gradientes, ya que antes la sumatoria iteraba sobre todo el vocabulario y ahora con Negative Sampling, iteramos sobre K muestras</font>


## PARTE 1g (2 point)

Repetiremos el ejercicio anterior, pero sin suponer que las $\tilde K$ muestras eran diferentes.
·Por sencillez de notación, nos referiremos como $\tilde w_1, \tilde w_2,...,\tilde w_K$ y a sus vectores como $\tilde u_1, ..., \tilde u_K$

En esta pregunta, no se puede suponer que las palabras sean diferentes, en otras palabras, $w_i=w_j$ podría ser cierto cuando $i=j$ lo sea. Notar que $o\notin\{w_1,...,w_K\}$. Para una palabra central $c$ una palabra de salida $o$, la función de coste Negative-Sampling está dada por:<br>

$$ J_{neg-sample}(v_c, o, U) = -log(σ(u_o^T v_c))−\sum_{k=1}^K log(σ(−u_k^T v_c)) $$

* Calcular la derivada parcial de $J_{neg-sample}$ respecto a la muestra negativa $\tilde u_k$.<br>
* Escribir las respuestas en términos de vecotres $v_c$ y $\tilde u_k$, donde $k\in[1,K]$.

Pista: Partir la sumatoria de la función de costo en dos partes, la suma sobre todas las palabras muestreadas igual a $\tilde u_k$ y la suma sobre todas las palabras muestreadas distintas de $\tilde u_k$.

<font color="green">Respuesta:<br>
$\begin{align}
\frac{\partial}{\partial \tilde u_k} J_{neg.\ sample}(v_c,o,U)
&=- \frac{\partial}{\partial \tilde u_k}
 log(\sigma(u_o^T v_c))
 -\frac{\partial}{\partial \tilde u_k}
 \sum_{k'=1}^{\tilde K} log(\sigma(-\tilde u_{k'}^T v_c))\\
&=-0 -\frac{\partial}{\partial \tilde u_k} \sum_{k'=1}^{\tilde K}
 (\delta (u_k=u_{k'})+\delta(u_k\neq u_{k'}))
 log(\sigma(-\tilde u_{k'}^T v_c))\\
&=-\frac{\partial}{\partial \tilde u_k} \sum_{k'=1}^{\tilde K}
 \delta(u_k=u_{k'})
 log(\sigma(-\tilde u_{k'}^T v_c))
 -\frac{\partial}{\partial \tilde u_k} \sum_{k'=1}^{\tilde K}
 \delta(u_k\neq u_{k'})
 log(\sigma(-\tilde u_{k'}^T v_c))\\
&=-\sum_{k'=1}^{\tilde K}
 \delta(u_k=u_{k'})
 \frac{\partial}{\partial \tilde u_k}
 log(\sigma(-\tilde u_k^T v_c))-0\\
&= -\sum_{k'=1}^{\tilde K}
 \delta(u_k=u_{k'})
 \frac{1}{\sigma(-\tilde u_k^T v_c)}
 \frac{\partial}{\partial \tilde u_k} \sigma(-\tilde u_k^T v_c)\\
&= -\sum_{k'=1}^{\tilde K}
 \delta(u_k=u_{k'})
 \frac{\sigma(-\tilde u_k^T v_c)(1-\sigma(-\tilde u_k^T v_c))}{\sigma(-\tilde u_k^T v_c)}
 \frac{\partial}{\partial \tilde u_k} (-\tilde u_k^T v_c)\\
&= \sum_{k'=1}^{\tilde K}
 \delta(u_k=u_{k'})
 (1-\sigma(-\tilde u_k^T v_c))\ v_c\\
&= (1-\sigma(-\tilde u_k^T v_c))\ v_c\ (\sum_{k'=1}^{\tilde K}
 \delta(u_k=u_{k'}))\\
\end{align}$
</font>

## PARTE 1h (3 points)
Suponga que la palabra central es $c=w_t$ y la ventana de contexto es
$[w_{t−m}, ..., w_{t−1}, w_t, w_{t+1}, ..., w_{t+m}]$, donde $m$ es el tamaño de la ventana. Recordar que para la versión skip-gram de word2vec, el costo total para esa ventana de contexto es
$$ J_{skip-gram}(v_c, w_{t−m}, ..., w_{t+m}, U) = \sum_{−m≤j≤m\atop j\neq 0} J(v_c, w_{t+j}, U) $$

Donde $J(v_c, w_{t+j}, U)$ representa un coste arbitrario en términos de la palabra central $c=w_t$ y la palabra de ventana $w{t+j}$.<br>
 $J(v_c, w_{t+j}, U)$ podría ser $J_{naive-softmax}(v_c, w_{t+j}, U)$ o $J_{neg-sample}(v_c, w_{t+j}, U)$, dependiendo de tu implementación.

Escribir las 3 derivadas parciales:
1. $\frac{\partial}{\partial U}J_{skip-gram}(v_c, w_{t−m}, ..., w_{t+m}, U)$
3. $\frac{\partial}{\partial v_c}J_{skip-gram}(v_c, w_{t−m}, ..., w_{t+m}, U)$
3. $\frac{\partial}{\partial v_w}J_{skip-gram}(v_c, w_{t−m}, ..., w_{t+m}, U)$ cuando $w\neq c$

·Escribir las respuestas en términos de $\frac{\partial}{\partial U}J(v_c, w_{t+j}, U)$ y $\frac{\partial}{\partial v_c}J(v_c, w_{t+j}, U)$.
Esto es muy sencillo, cada solución debe ser una línea.

<font color="green">Respuesta<br>
$\begin{align}
\frac{\partial}{\partial U}J_{skip-gram}(v_c, w_{t−m}, ..., w_{t+m}, U)
&=\frac{\partial}{\partial U}
\sum_{-m\le j\le m\atop j\neq 0} J(v_c, w_t+j, U)\\
&=\sum_{-m\le j\le m\atop j\neq 0}
\frac{\partial}{\partial U} J(v_c, w_t+j, U)\\
\end{align}$

$\begin{align}
\frac{\partial}{\partial v_c}J_{skip-gram}(v_c, w_{t−m}, ..., w_{t+m}, U)
&=\frac{\partial}{\partial v_c}
\sum_{-m\le j\le m\atop j\neq 0} J(v_c, w_t+j, U)\\
&=\sum_{-m\le j\le m\atop j\neq 0}
\frac{\partial}{\partial v_c} J(v_c, w_t+j, U)\\
\end{align}$

$\begin{align}
\frac{\partial}{\partial v_w}J_{skip-gram}(v_c, w_{t−m}, ..., w_{t+m}, U)
&=\frac{\partial}{\partial v_w}
\sum_{-m\le j\le m\atop j\neq 0} J(v_c, w_t+j, U)\\
&=\sum_{-m\le j\le m\atop j\neq 0}
\frac{\partial}{\partial v_w} J(v_c, w_t+j, U)\\
&=0
\end{align}$
</font>

# PARTE 2 Código (18 points)
En esta parte implementarás el modelo word2vec y entrenarás tus propios word vectors mediante Stochastic Gradient Descent (SGD).
Antes de comenzar, se deben ejecutar las siguientes instrucciones dentro del directorio del proyecto con el fin de crear el apropiado entorno virtual para conda.
 Esto garantiza que tienes todos las librerías necesarias para completar esta parte.
·Obs. Debes terminar la sección matemática anterior antes de intentar implementar el código ya que se pedirá implementar estas funciones matemáticas.
También querrás implementar y testear la siguientes secciones en orden ya que son acumulativas.

` conda env create -f env.yml `<br>
` conda activate a2 `

Al terminar con el ejercicio, puedes desactivas este entorno con <br>
` conda deactivate `<br>

Para cada método, en los comentarios hemos incluído aproximadamente la cantidad de líneas que nuestra solución tiene para que tomes de referencia.
 Los bucles for en python toman más tiempo en finalizar cuando se utilizan sobre arrays grandes, por eso esperamos que utilices los métodos de numpy. Estaremos controlando la eficiencia de tu código, serás capaz de ver los resultados del autograder cuando cargues el código al Gradescope. Se recomienda publicarlo temprana y continuamente.

In [ ]:
#Imports

## PARTE 2a  (12 points)
Empezaremos implementando los métodos dentro de **word2vec.py**

Puedes testear un método $m$ particular ejecutando
` python word2vec.py m `<br>
Por ejemplo, puedes testear la función sigmoidea ejecutando: <br>
` python word2vec.py sigmoid `

Ejercicios:
1. Implementar el método **sigmoid**, que recive un vector y le aplica la función sigmoidea.
2. Implementar la función de coste softmax y gradiente en el método **naiveSoftmaxLossAndGradient**
3. Implement the negative sampling loss and gradient in the negSamplingLossAndGradient
method.
4. Implementar el modelo skip-gram en el método **skipgram**.
Al finalizar, verificar la implementación entera ejecutando <br>
` python word2vec.py `

In [28]:
#code 2a
!python word2vec.py sigmoid
!python word2vec.py naiveSoftmaxLossAndGradient
!python word2vec.py negSamplingLossAndGradient
!python word2vec.py skipgram

=== Sanity check for sigmoid ===
Tests for sigmoid passed!
==== Gradient check for naiveSoftmaxLossAndGradient ====
Gradient check passed!. Read the docstring of the `gradcheck_naive` method in utils.gradcheck.py to understand what the gradient check does.
Gradient check passed!. Read the docstring of the `gradcheck_naive` method in utils.gradcheck.py to understand what the gradient check does.
==== Gradient check for negSamplingLossAndGradient ====
Gradient check passed!. Read the docstring of the `gradcheck_naive` method in utils.gradcheck.py to understand what the gradient check does.
Gradient check passed!. Read the docstring of the `gradcheck_naive` method in utils.gradcheck.py to understand what the gradient check does.
==== Gradient check for skip-gram with naiveSoftmaxLossAndGradient ====
Gradient check passed!. Read the docstring of the `gradcheck_naive` method in utils.gradcheck.py to understand what the gradient check does.
======Skip-Gram with naiveSoftmaxLossAndGradient Te

## PARTE 2b  (4 points)
Complete la implementación para el optimizador SGD en el método **sgd.py**. Verificar tu implementación ejecutando <br>
` python sgd.py `

In [30]:
#code 2b
!python sgd.py

Running sanity checks...
iter 100: 0.004578
iter 200: 0.004353
iter 300: 0.004136
iter 400: 0.003929
iter 500: 0.003733
iter 600: 0.003546
iter 700: 0.003369
iter 800: 0.003200
iter 900: 0.003040
iter 1000: 0.002888
test 1 result: 8.414836786079764e-10
iter 100: 0.000000
iter 200: 0.000000
iter 300: 0.000000
iter 400: 0.000000
iter 500: 0.000000
iter 600: 0.000000
iter 700: 0.000000
iter 800: 0.000000
iter 900: 0.000000
iter 1000: 0.000000
test 2 result: 0.0
iter 100: 0.041205
iter 200: 0.039181
iter 300: 0.037222
iter 400: 0.035361
iter 500: 0.033593
iter 600: 0.031913
iter 700: 0.030318
iter 800: 0.028802
iter 900: 0.027362
iter 1000: 0.025994
test 3 result: -2.524451035823933e-09
----------------------------------------
ALL TESTS PASSED
----------------------------------------


## PARTE 2c (2 points)
Hora de Exposición! Es la hora de cargar datos reales y entrenar los word vectors con todo ya implementado! Utilizaremos el DataSet **Stanford Sentiment Treebank (SST)** para entrenar los word vectors, y luego aplicarlos en una simple tarea de análisis del sentimiento.<br>

Para obtener el dataset, ejecutar:<br>
` sh get datasets.sh `<br>
 No hay código adicional para escribir en esta parte, solo ejecutar: <br>
` python run.py `

Obs.: El proceso de entrenamiento podría llevar mucho tiempo dependiendo de la eficiencia de tu implementación y el poder de cálculo de tu computadora (a una implementación eficiente le lleva entre 1 y 2 horas). Planificar acorde!

Luego de 40,000 iteraciones, el script terminará y aparecerá un gráfico para tus word vectors.
Además, se guardará como **word_vectors.png** en tu directorio. Incluir el gráfico en tu documento escrito, explicando brevemente al menos 3 oraciones que veas en el gráfico.

In [33]:
!get_datasets.sh

In [1]:
#code 2c
!python run.py

Traceback (most recent call last):
  File "D:\Estudio\Stanford\NLP with Deep Learning, Winter 2019\Tps\a2\run.py", line 6, in <module>
    import matplotlib
ModuleNotFoundError: No module named 'matplotlib'
